In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig
from functions import *

In [2]:
add_special_tokens = False
batch_size = 4
max_length = 1024

In [3]:
# MODEL_PATH = "/kaggle/input/gemma/transformers/7b-it/2"
MODEL_PATH = "google/gemma-7b-it"
# Found a good blog to catch me up fast!
# https://huggingface.co/blog/4bit-transformers-bitsandbytes
# https://huggingface.co/docs/transformers/v4.38.1/en/quantization#compute-data-type
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map = "auto",
    trust_remote_code = True,
#     torch_dtype=torch.float16,
    quantization_config=quantization_config,
)

# model = model.to_bettertransformer()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
train = pd.read_csv('/home/zhenlan/Desktop/Projects/LLM Prompt Recovery/Data/nbroad-v1.csv')

# encode prompt, text
prompts_txt = list(train.rewrite_prompt.unique())
prompts = tokenizer.batch_encode_plus(prompts_txt,add_special_tokens=add_special_tokens,return_attention_mask=False)['input_ids']
n_prompt = len(prompts)
original_texts = tokenizer.batch_encode_plus(train.original_text.tolist(),add_special_tokens=add_special_tokens,\
                                             return_attention_mask=False,truncation=True, max_length=max_length)['input_ids']
rewritten_texts = tokenizer.batch_encode_plus(train.rewritten_text.tolist(),add_special_tokens=add_special_tokens,\
                                              return_attention_mask=False,truncation=True, max_length=max_length)['input_ids']

In [5]:
token_ids, rewritten_text_len = create_batch_txt(original_texts, rewritten_texts, prompts, tokenizer)
CE_out = gen_CE(token_ids, rewritten_text_len, batch_size, tokenizer, model)
CE_out = np.array(CE_out).reshape(train.shape[0],n_prompt)
prompt_idx = CE_out.argmin(-1)

OutOfMemoryError: CUDA out of memory. Tried to allocate 5.89 GiB. GPU 0 has a total capacity of 23.64 GiB of which 4.47 GiB is free. Including non-PyTorch memory, this process has 18.89 GiB memory in use. Of the allocated memory 14.15 GiB is allocated by PyTorch, and 3.30 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
np.save('CE_out.npy', CE_out)